# Import von WIAG-Daten in FactGrid
Formatiere CSV-Daten aus WIAG als V1-Statements für QuickSteps  
siehe [Einführung in QuickStatements](https://www.wikidata.org/wiki/Help:QuickStatements)
und die [Properties des FactGrid](https://database.factgrid.de/wiki/FactGrid:Directory_of_Properties)

Inhalt:

- [neue Objekte in FactGrid anlegen](#neue-Objekte-in-FactGrid-anlegen)

In [2]:
using CSV, DataFrames, Dates

In [33]:
input_path = "C:\\Users\\georg\\Documents\\projekte-doc\\Forschungsdaten-GS\\data\\FactGrid"

"C:\\Users\\georg\\Documents\\projekte-doc\\Forschungsdaten-GS\\data\\FactGrid"

In [34]:
output_path = input_path

"C:\\Users\\georg\\Documents\\projekte-doc\\Forschungsdaten-GS\\data\\FactGrid"

## neue Objekte in FactGrid anlegen
Es wird davon ausgegangen, dass die betreffenden Personen noch nicht in FactGrid vorhanden sind. Die Spalte "FactGrid_ID" sollte also für alle Einträge leer sein.

Daten aus WIAG einlesen: Quelle Export "CSV Personendaten"

In [4]:
input_file = "WIAG-can_2023-12-01.csv"
input_path_file = joinpath(input_path, input_file)

"C:\\Users\\georg\\Documents\\projekte-doc\\WIAGweb3\\data\\FactGrid\\WIAG-can_2023-12-01.csv"

In [23]:
df_person = CSV.read(input_path_file, DataFrame);
nrow(df_person)

15141

Benenne die Spalten um entsprechend den Konventionen des FactGrid

In [24]:
columns = [
    :displayname => :Lde,
    :description => :Dde,
    :date_of_birth => :P77,
    :date_of_death => :P38,
    :GND_ID => :P76,
    :GSN => :P472,
    :id => :P601
];

In [25]:
rename!(df_person, columns);

Kopiere den Label in Deutsch auch als Label in Englisch

In [26]:
df_person.Len = df_person.Lde;

Füge Daten ein, die für alle Personen gleich sind:
Mensch, Teil der Germania Sacra Forschungsdaten, männlich

In [27]:
insertcols!(df_person, 
    :P2 => "Q7",
    :P131 => "Q153178",
    :P154 => "Q18"
);    

Reduziere die Daten auf die Spalten, die in FactGrid übernommen werden sollen.

In [43]:
drop_columns = [
    :givenname,
    :prefix,
    :familyname,    
    :Wikidata_ID,
    :Wikipedia
];

In [44]:
df_fq = select(df_person, Not(drop_columns));

In [45]:
df_fq[50:52, :]

Row,P601,Lde,P77,P38,Dde,P76,P472,FactGrid_ID,Len,P2,P131,P154
,String31,String,String15?,String31?,String,String15?,String15?,String7?,String,String,String,String
1,WIAG-Pers-CANON-80952-001,Christoph von Knöringen,missing,missing,"~ 1499, Domscholaster Domstift Augsburg 1499",missing,007-00810-001,missing,Christoph von Knöringen,Q7,Q153178,Q18
2,WIAG-Pers-CANON-84019-001,Friedrich von Leonrod,1483,1539,"* 1483, + 1539, Domherr Domstift Augsburg 1514, Domherr Domstift Eichstätt 1501",missing,054-01638-001,missing,Friedrich von Leonrod,Q7,Q153178,Q18
3,WIAG-Pers-CANON-49717-001,Eustachius von Stain,missing,1523,"+ 1523, Domherr Domstift Eichstätt 1512-1517, Domherr Domstift Augsburg 1506-1523",missing,missing,missing,Eustachius von Stain,Q7,Q153178,Q18


Definiere Umwandlungsfunktionen  
*offen:* Datumsangaben sind noch zu besprechen, wegen der Verarbeitung von unscharften Zeitangaben.

In [13]:
fqs_string(s) = "\"" * s * "\""

fqs_string (generic function with 1 method)

In [14]:
function create(out, row, col_list) 
    println(out, "CREATE")    
    for col in col_list
        println(out, "LAST\t", string(col), "\t", fqs_string(row[col]))        
    end
end

create (generic function with 1 method)

In [50]:
"""
    set_property_list (out, row, object, col_list, fmt_list)

    

# Arguments
- `out::IOStream`: output stream
- `row::DataFrameRow`: input data
- `object`: FactGrid identifier or "LAST"
- `col_list`: list of colums where data should be read from
- `fmr_list`: list of formatting functions (one for each column)
"""
function set_property_list(out, row, object, col_list, fmt_list)
    for (col, fmt) in zip(col_list, fmt_list)
        if (!ismissing(row[col]))
            println(out, object, "\t", string(col), "\t", fmt(row[col]))
        end
    end
end

set_property_list

Gib alle Elemente in `df_fq` aus. Falls es schon eine Datei mit gleichem Namen im angegebenen Verzeichnis gibt, wird die Datei überschrieben.

In [40]:
date_key = Dates.format(now(), "yyyy-mm-dd")

"2023-12-01"

In [47]:
output_file = "Insert_WIAG-can_" * date_key * "_FQ.csv"
output_path_file = joinpath(output_path, output_file)

"C:\\Users\\georg\\Documents\\projekte-doc\\Forschungsdaten-GS\\data\\FactGrid\\Insert_WIAG-can_2023-12-01_FQ.csv"

In [48]:
open(output_path_file, "w") do out_stream
    for row in eachrow(df_fq)
        create(out_stream, row, [:Lde, :Len, :Dde])
        set_property_list(out_stream, row, "LAST", 
            [:P2, :P131, :P154, :P601, :P76, :P472], 
            [identity, identity, identity, fqs_string, fqs_string, fqs_string])
    end        
end